In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import sys
sys.path.append('/home/diego/Trabajo/Proyectos/Pynterpred_devel/')
import numpy as np
import mdtraj
import pynterpred as pnt
import nglview
from tqdm import tqdm_notebook as tqdm

from simtk import openmm, unit
from simtk.openmm import app

In [3]:
receptor = pnt.Receptor('testsystems/Barnase-Barstar/pdbs/Barnase.pdb','amber14-all.xml',pH=7.0)
ligand   = pnt.Ligand('testsystems/Barnase-Barstar/pdbs/Barstar.pdb','amber14-all.xml',pH=7.0)

In [4]:
context=pnt.MMContext(receptor,ligand)

In [5]:
region = pnt.Region()

In [6]:
region.centers_in_layer("regular_cartesian",receptor,ligand,delta_x=0.6) #delta_x=0.25

In [7]:
region.rotators_in_quaternions_region(nside=2) #nside=8

In [ ]:
for qrotor in tqdm(region.qrotors):
    context.rotate_ligand(qrotor)
    for center in tqdm(region.centers):
        context.center_ligand(center*unit.nanometer)
        context.get_potential_energy()

In [9]:
for center in tqdm(region.centers):
    context.center_ligand(center*unit.nanometer)
    context.get_potential_energy()